### MeOffendEs 2021
### Convertir los datasets de las competencias a json y separarlos 75/25 train/test
- Se crea el modelo evomsa pre-entrenado exist2021_Es.evomsa
- En Single f1_score_macro  = 0.7566480556897693
- En Muliple f1_score_macro = 
  - haha2018, mexa3t, misoginia
- La mejora es apenas 0.003 0.01

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# MeOffendEs2021

df = pd.read_csv("../meoffendes2021_train-non-contextual.csv", usecols=['text', 'klass']) #,   header=0, nrows=50)# , nrows=30, header=1)
# df.rename(columns = {'task1':'klass'}, inplace = True)

In [3]:
df.shape

(5060, 2)

In [4]:
df

text  klass
0     Soy el Clint Eastwood de los Puentes de Madiso...      0
1     Actualmente ya pasó de moda la pucha joto, aho...      1
2     ¿Es cierto esto? Y no me refiero a lo que dijo...      0
3     Vuela pega y esquiva... la neta está de la ver...      0
4     Mejor puto disfraz de la noche!!!! 👊👊👊Por terc...      0
...                                                 ...    ...
5055  @USUARIO Alv, bien salvado el pino. Sólo tengo...      0
5056  Verga estoy viendo que un compañero que tuve e...      0
5057  Firmado por toda la pandilla de @USUARIO a tod...      0
5058  Pinshe profe se mamo con el examen  Todavía qu...      0
5059                           @USUARIO La piel prieta?      1

[5060 rows x 2 columns]

In [5]:
np.unique(df['klass'])
# 0 es ofensivo
# 1 No es ofensivo

array([0, 1], dtype=int64)

In [6]:
# df["klass"] = np.where(df["klass"] == "sexist", 1, 0)
# sexist = 1
# No sexist = 

In [7]:
#df

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['klass'], train_size=0.75)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3795,), (1265,), (3795,), (1265,))

In [9]:
type(X_train)

pandas.core.series.Series

In [10]:
df_train = pd.concat([X_train, y_train], axis=1)
print(df_train.shape)
df_train

(3795, 2)


text  klass
2348  @USUARIO @USUARIO Que rico mamas el camote tie...      0
1944  Ampliación: La salida a final de mes del juego...      0
3981  !es verdad! !luis le manda un saludo a una chi...      0
430   Me emputa la gente ardida que se estanca y no ...      1
2899  No estoy loca 😜.  Mi realidad es diferente a l...      0
...                                                 ...    ...
3027  Dicen que estoy loca porqué no uso chamarra co...      0
2433  No colega, no las conozco ¿me las podría compa...      0
3053  T-3horas para el DRAFT de la liga más cabrona ...      0
4760  Ustedes si puden andar de chanceros, pero cuid...      0
1459       @USUARIO Seguro lo dice pura gente ardida!!!      0

[3795 rows x 2 columns]

In [11]:
df_test = pd.concat([X_test, y_test], axis=1)
print(df_test.shape)
df_test

(1265, 2)


text  klass
1661  ¡QUE NO JOTO, QUE NO! 🙄 Si ya tenías mi follow...      0
3837  Cómo me encanta la verdad q me den así no me l...      0
1426  Que triste que andes pendiente de papá zorro m...      0
4869  Mejor donen culeros, putas ratas tu y el pende...      1
1191  Ganando como putas siempre con mis gordas ador...      0
...                                                 ...    ...
334   Si pero porque hay lameculos en la coparmex, s...      1
1164  Hasta cuando este orgullo me deténdra a decirt...      1
2264  Tanto que lucharon los dreamers por daca para ...      1
947   ya dejen de mamar con sus chistes malos de los...      0
3689  @USUARIO @USUARIO @USUARIO Jajajajajjaja ese j...      0

[1265 rows x 2 columns]

In [12]:
with open('..\meoffendes2021_Es_train.json', 'w') as f:
    f.write(df_train.to_json(orient='records', lines=True))
    
with open('..\meoffendes2021_Es_test.json', 'w') as f:
    f.write(df_test.to_json(orient='records', lines=True))

In [13]:
from EvoMSA.base import EvoMSA
from microtc.utils import save_model
from sklearn import metrics
from microtc.utils import load_model
from microtc.utils import save_model

In [14]:
evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, models=None)

# Pq a .squeeze() ? 

In [15]:
evo.fit(X_train.squeeze(), y_train.squeeze())
pred = evo.predict(X_test.squeeze())
print(metrics.f1_score(y_test, pred, average="macro"))

100%|██████████| 30/30 [00:21<00:00,  1.42it/s]

0.7566480556897693


In [16]:
# crear modelo pre-entrenado en Sigle par evitar efecto cascada
save_model(evo, '..\\..\\single_models\\meoffendes21_single_Es.evomsa') 

### Multiple
-----

In [ ]:
stacked_method = "sklearn.svm.LinearSVC"
# usar = "sklearn.naive_bayes.BernoulliNB"
# usar = "sklearn.naive_bayes.MultinomialNB"

haha18_model = [load_model('..\\..\\models\\haha2018_Es.evomsa'), stacked_method]
mexa3t_model = [load_model('..\\..\\models\\mexa3t2018_aggress_Es.evomsa'), stacked_method]
misoginia_model = [load_model('..\\..\\models\\misoginia_Es.evomsa'), stacked_method]

exist21_model = [load_model('..\\..\\models\\exist2021_Es.evomsa'), stacked_method]
haha21_model = [load_model('..\\..\\models\\haha2021_Es.evomsa'), stacked_method]

mm = [haha_model, mexa3t_model, misoginia_model]

evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, stacked_method=stacked_method, models= mm)

In [ ]:
evo.fit(X_train.squeeze(), y_train.squeeze())
pred = evo.predict(X_test.squeeze())
print(metrics.f1_score(y_test, pred, average="macro"))

In [ ]:
# crear modelo pre-entrenado
save_model(evo, '..\\..\\models\\exist2021_Es.evomsa') 

In [ ]:
# probarlo
_ = load_model('..\\..\\models\\exist2021_Es.evomsa') 
pred = _.predict(X_test.squeeze())
print(metrics.f1_score(y_test, pred, average="macro"))